<a href="https://colab.research.google.com/github/afnanhabib787/Artificial-Intelligence/blob/main/Apple_v_Orange_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"afnanhabib787","key":"30b2e622f6c5384f87a3c6ea1e6a3cab"}'}

In [ ]:
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d balraj98/apple2orange-dataset

 98% 73.0M/74.8M [00:01<00:00, 43.1MB/s]
100% 74.8M/74.8M [00:01<00:00, 57.3MB/s]


In [ ]:
from zipfile import ZipFile
file_name = 'apple2orange-dataset.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_gen = ImageDataGenerator(rescale = 1.0/255)

In [ ]:
train_data = train_data_gen.flow_from_directory(
    "/content/train_data",
    target_size=(150,150),
    batch_size=30,
    class_mode='binary'
)

Found 2014 images belonging to 2 classes.


In [ ]:
train_data.class_indices


{'trainA': 0, 'trainB': 1}

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [ ]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=[['acc']])

In [ ]:
history = model.fit_generator(
    train_data,
    epochs=30
    )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
68/68 [==============================] - 90s 1s/step - loss: 0.3205 - acc: 0.8640
Epoch 2/30
68/68 [==============================] - 90s 1s/step - loss: 0.2448 - acc: 0.9032
Epoch 3/30
68/68 [==============================] - 91s 1s/step - loss: 0.2266 - acc: 0.9186
Epoch 4/30
68/68 [==============================] - 90s 1s/step - loss: 0.2021 - acc: 0.9220
Epoch 5/30
68/68 [==============================] - 90s 1s/step - loss: 0.1952 - acc: 0.9315
Epoch 6/30
68/68 [==============================] - 90s 1s/step - loss: 0.1833 - acc: 0.9265
Epoch 7/30
68/68 [==============================] - 90s 1s/step - loss: 0.1719 - acc: 0.9325
Epoch 8/30
68/68 [==============================] - 90s 1s/step - loss: 0.1577 - acc: 0.9389
Epoch 9/30
68/68 [==============================] - 90s 1s/step - loss: 0.1576 - acc: 0.9444
Epoch 10/30
68/68 [==============================] - 90s 1s/step - loss: 0.1464 - acc: 0.9489
Epoch 11/30
68/68 [==============================] - 90s 1s/step - lo

In [23]:
test_data = train_data_gen.flow_from_directory(
    "/content/test_data",
    target_size=(150,150),
    batch_size=30,
    class_mode='binary'
)

Found 514 images belonging to 2 classes.


In [25]:
test_data.class_indices

{'testA': 0, 'testB': 1}

In [46]:
model.evaluate(test_data)

18/18 [==============================] - 7s 362ms/step - loss: 0.2462 - acc: 0.9358


[0.24616359174251556, 0.9357976913452148]

In [40]:
from keras.preprocessing import image
img = image.load_img('/content/test_data/testA/n07740461_10011.jpg', target_size=(150,150))

In [41]:
x = image.img_to_array(img)

In [42]:
x.shape

(150, 150, 3)

In [43]:
x = x.reshape((1,) + x.shape)

In [44]:
x.shape

(1, 150, 150, 3)

In [45]:
model.predict(x)

array([[1.]], dtype=float32)

In [ ]:
model.predict(test_data)

In [ ]:
import numpy as np

np.where(model.predict(test_data)>0,"Orange","Apple")

In [50]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/train_data',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

Found 2014 images belonging to 2 classes.


In [51]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(learning_rate=1e-4),
metrics=['acc'])

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)      

In [54]:
history = model.fit_generator(
    train_generator,
    epochs=20,
    )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
63/63 [==============================] - 99s 2s/step - loss: 0.1719 - acc: 0.9340
Epoch 2/20
63/63 [==============================] - 101s 2s/step - loss: 0.1806 - acc: 0.9394
Epoch 3/20
63/63 [==============================] - 102s 2s/step - loss: 0.1770 - acc: 0.9295
Epoch 4/20
63/63 [==============================] - 101s 2s/step - loss: 0.1691 - acc: 0.9374
Epoch 5/20
63/63 [==============================] - 103s 2s/step - loss: 0.1761 - acc: 0.9305
Epoch 6/20
63/63 [==============================] - 105s 2s/step - loss: 0.1700 - acc: 0.9335
Epoch 7/20
63/63 [==============================] - 104s 2s/step - loss: 0.1677 - acc: 0.9330
Epoch 8/20
63/63 [==============================] - 102s 2s/step - loss: 0.1664 - acc: 0.9330
Epoch 9/20
63/63 [==============================] - 102s 2s/step - loss: 0.1671 - acc: 0.9330
Epoch 10/20
63/63 [==============================] - 103s 2s/step - loss: 0.1649 - acc: 0.9374
Epoch 11/20
63/63 [==============================] - 103s 2s

In [55]:
model.evaluate(test_data)

18/18 [==============================] - 7s 370ms/step - loss: 0.1701 - acc: 0.9319


[0.17010512948036194, 0.9319066405296326]

In [56]:
model.predict(test_data)

array([[4.60147858e-04],
       [9.13008407e-05],
       [6.90636039e-03],
       [1.20472908e-03],
       [9.48328376e-01],
       [5.11600156e-05],
       [1.26625389e-01],
       [1.01712840e-05],
       [9.95967686e-01],
       [9.98792887e-01],
       [9.41830158e-01],
       [1.65949166e-02],
       [2.89976597e-04],
       [8.87101889e-03],
       [9.32579160e-01],
       [5.72468996e-01],
       [2.11174793e-05],
       [9.90844011e-01],
       [4.69976395e-01],
       [2.08482742e-02],
       [9.97863114e-01],
       [8.65380287e-01],
       [8.17477703e-04],
       [5.18491864e-03],
       [9.93827462e-01],
       [8.03864002e-03],
       [1.98485851e-02],
       [3.87827069e-01],
       [1.50388479e-03],
       [9.05410528e-01],
       [9.98593450e-01],
       [9.95708466e-01],
       [9.74859178e-01],
       [8.29310920e-06],
       [1.22707933e-01],
       [8.77884150e-01],
       [2.93704569e-02],
       [8.59173119e-01],
       [1.32450759e-02],
       [9.46462154e-04],


In [69]:
import numpy as np

a =np.where(model.predict(test_data)>0.5,"Orange","Apple")